In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from flipside import Flipside
flipside = Flipside("023b83a2-e411-4849-a961-3742c87bc9c9", "https://api-v2.flipsidecrypto.xyz")

We first set the name of the liquidity pool we want to query data from.

In [2]:
pool_name = "USDC-WETH 100 1"

This write the sql code and start query.

In [3]:
sql = """
SELECT TX_HASH, NFT_ADDRESS, BLOCK_TIMESTAMP, BLOCK_NUMBER, NFT_FROM_ADDRESS, NFT_TO_ADDRESS, TOKENID
FROM ethereum.core.ez_nft_transfers
WHERE PROJECT_NAME = 'uniswap' AND
      NFT_ADDRESS IN (SELECT DISTINCT NF_POSITION_MANAGER_ADDRESS
                      FROM ethereum.uniswapv3.ez_lp_actions
""" +\
"WHERE POOL_NAME = '" + pool_name + "')" 

query_result_set = flipside.query(sql, page_number = 1, page_size = 30000)

current_page_number = 1
page_size = 30000
total_pages = 2

# All the data will be stored in a list called "all_rows"
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

# Merge all rows to a dataframe
df = pd.DataFrame(all_rows)
df.drop(columns = ['__row_index'], inplace = True)
df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])
df.head(10)

,tx_hash,nft_address,block_timestamp,block_number,nft_from_address,nft_to_address,tokenid
0,0xbc1e8cfadc0f4ed4197b3726ba14b4442ed962e765dc...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2022-05-25 11:08:51+00:00,14841534,0x0000000000000000000000000000000000000000,0x4954c56c0fda0a30fb88c85a82aa6b3faadc5d1c,244733
1,0x493a3d8edec24cee4e085369c2f5e9d84ca763587127...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2022-06-17 03:26:59+00:00,14976966,0x000000004b60a12164e6942dcd3e007f079b725b,0xe34139463ba50bd61336e0c446bd8c0867c6fe65,249787
2,0x6c23308548f01aaa24134a29558d3a99b8c7d8f8fb80...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-12-13 23:06:07+00:00,13799762,0x4d145a5e90736ad71cbf0081366625bd9eb170cc,0x1f98407aab862cddef78ed252d6f557aa5b0f00d,166672
3,0x9a956bbc247225f5cc0c5c2287ba9056b0106f29a456...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-12-13 15:01:18+00:00,13797521,0x0000000000000000000000000000000000000000,0xe2f56d087e4c2f3fb333a9b198ed6779dcf1c9e8,166560
4,0x42827db9ce09012639dd56e414f16e6f103cce98e60f...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-12-13 20:35:51+00:00,13799081,0x0000000000000000000000000000000000000000,0xabf85ddabd073a2f662e020fdc740a5cd3acf000,166636
5,0x34b0898939dc03989b849e43cff7e341308d8a3223b3...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-12-13 22:41:49+00:00,13799654,0x0000000000000000000000000000000000000000,0x4ce6aea89f059915ae5efbf34a2a8adc544ae09e,166666
6,0x91977840b957f394dbbb3817ed17d569e893414592ae...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-12-13 18:16:51+00:00,13798451,0x3ef26504dbc8dd7b7aa3e97bc9f3813a9fc0b4b0,0x1e4bda16734c1f9e86a66e310fcf8bd3c4f9c0f0,100878
7,0x900fdb7f243faf29079f2a01b45c0c60b84638e16268...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-12-14 07:59:03+00:00,13802067,0x0000000000000000000000000000000000000000,0xc1a2f762f67af72fd05e79afa23f8358a4d7dbaf,166823
8,0xba98ed0d79d62927c39706efac80c386039ce323a526...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-12-14 14:06:38+00:00,13803711,0x0000000000000000000000000000000000000000,0xae667ed58c0d9198fc0b9261156d48296c1bb3da,166941
9,0xe54f6c7f499a24d5b76b14052b7c7b27664374198597...,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-12-14 05:09:09+00:00,13801354,0x0000000000000000000000000000000000000000,0xf99f2df8fb2b8b873cfbd316296ba82ff55e03db,166780


Lastly, we want to save the data in our desire repository.

In [4]:
df.to_csv("/Users/zhicong/Dropbox/DEXs/uniswapv3_NFT/" + pool_name + ".csv", index=False)